### Without cls load

In [33]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
import sys
import numpy as np

sys.path.append("../")

warnings.filterwarnings("ignore")

In [34]:
import os

data_dir = "~/ae/work/data/training_data_v10/processed/"
train_path = data_dir + "brexit_parsed.csv"
valid_path = data_dir + "asia_bibi_parsed.csv"
model_dir = "/home/eartemov/ae/work/models/multilingual_L-12_H-768_A-12"
# init_checkpoint_pt = os.path.join(model_dir, "pytorch_model.bin")
init_checkpoint_pt = "/home/eartemov/ae/work/models/bert-base-multilingual-cased.tar.gz"
bert_config_file = os.path.join(model_dir, "bert_config.json")
# vocab_file = os.path.join(model_dir, "vocab.txt")
token = "96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729"
vocab_file = "/home/eartemov/models/bert-base-multilingual-cased/" + token

In [35]:
from modules import BertNerData as NerData

In [36]:
data = NerData.create(train_path, valid_path, vocab_file, is_cls=False, batch_size=16)

In [37]:
len(data.tokenizer.vocab)

119547

In [38]:
import torch

In [39]:
torch.__version__

'0.4.1'

In [40]:
torch.cuda.set_device(0)

In [41]:
from modules.models.bert_models import BertBiLSTMAttnNMTJoint, BertBiLSTMAttnNCRF, BertBiLSTMAttnNCRFJoint

In [42]:
# model = BertBiLSTMAttnNCRFJoint.create(
#          len(data.label2idx), len(data.cls2idx), bert_config_file, init_checkpoint_pt,
#          enc_hidden_dim=1024, rnn_layers=1, num_heads=6, input_dropout=0.5, nbest=11)

In [43]:
model = BertBiLSTMAttnNCRF.create(
        len(data.label2idx), bert_config_file, init_checkpoint_pt,
       enc_hidden_dim=1024, rnn_layers=1, num_heads=6, input_dropout=0.5, nbest=11)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpmla39agv
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

build CRF...


In [44]:
model.get_n_trainable_params()

7370256

In [45]:
from modules import NerLearner

In [46]:
num_epochs = 150
learner = NerLearner(model, data,
                     best_model_path="/home/eartemov/ae/work/models/AGRR-2019/slavic_without_clf.cpt",
                     # best_model_path="/home/eartemov/ae/work/models/slavic-base.cpt",
                     lr=0.0001, clip=1.0, sup_labels=data.id2label[1:],
                     t_total=num_epochs * len(data.train_dl))

In [47]:
# learner.fit(num_epochs, target_metric="f1")

In [48]:
learner.load_model()

In [49]:
from modules.data.bert_data import get_bert_data_loader_for_predict

In [50]:
dl = get_bert_data_loader_for_predict(valid_path, learner)

In [51]:
# preds_res, preds_cls = learner.predict(dl)
preds_res = learner.predict(dl)

In [52]:
from sklearn_crfsuite.metrics import flat_classification_report

In [53]:
from modules.utils.utils import bert_labels2tokens, first_choicer

In [54]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds_res)
true_tokens, true_labels = bert_labels2tokens(dl, [x.labels for x in dl.dataset])

In [55]:
len(true_tokens)

5457

In [56]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels)

#### Without report

In [90]:
print(tokens_report)

              precision    recall  f1-score   support

       I_EVT       0.45      0.39      0.42        54
       I_LOC       0.67      0.90      0.77      1422
         I_O       0.99      1.00      0.99     72152
       I_ORG       0.85      0.60      0.70      1876
       I_PER       0.90      0.77      0.83      4121
       I_PRO       0.37      0.60      0.46       233

    accuracy                           0.97     79858
   macro avg       0.70      0.71      0.70     79858
weighted avg       0.97      0.97      0.97     79858



In [91]:
from sklearn.metrics import accuracy_score, f1_score

pl = []
for line in pred_labels:
    for p in line:
        pl.append(p)
        
tl = []
for line in true_labels:
    for p in line:
        tl.append(p)
accuracy_score(tl, pl), f1_score(tl, pl, average="macro")

(0.9715244559092389, 0.6951485358141007)

In [92]:
true_cls = [x.cls for x in dl.dataset]

In [93]:
print(flat_classification_report([true_cls], [preds_cls]))

              precision    recall  f1-score   support

          bg       0.95      1.00      0.97      1288
          cs       1.00      0.98      0.99      1170
          pl       0.96      1.00      0.98      1456
          ru       1.00      0.94      0.97      1543

    accuracy                           0.98      5457
   macro avg       0.98      0.98      0.98      5457
weighted avg       0.98      0.98      0.98      5457



In [94]:
from modules.utils.utils import tokens2spans

In [95]:
pred_spans = tokens2spans(pred_tokens, pred_labels)

In [96]:
true_spans = tokens2spans(true_tokens, true_labels)

In [97]:
from modules.utils.plot_metrics import get_bert_span_report

In [98]:
from modules.utils.utils import voting_choicer

In [99]:
print(get_bert_span_report(dl, preds_res, fn=first_choicer))

              precision    recall  f1-score   support

         PER      0.769     0.688     0.726      2846
         LOC      0.642     0.872     0.739      1477
         ORG      0.692     0.623     0.656       984
         EVT      0.297     0.407     0.344        27
         PRO      0.332     0.581     0.423       167

   micro avg      0.688     0.721     0.704      5501
   macro avg      0.546     0.634     0.578      5501
weighted avg      0.706     0.721     0.706      5501



### With cls load

In [66]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
import sys
import numpy as np

sys.path.append("../")

warnings.filterwarnings("ignore")

In [67]:
import os

data_dir = "~/ae/work/data/training_data_v10/processed/"
train_path = data_dir + "brexit_parsed.csv"
valid_path = data_dir + "asia_bibi_parsed.csv"
model_dir = "/home/eartemov/ae/work/models/multilingual_L-12_H-768_A-12"
# init_checkpoint_pt = os.path.join(model_dir, "pytorch_model.bin")
init_checkpoint_pt = "/home/eartemov/ae/work/models/bert-base-multilingual-cased.tar.gz"
bert_config_file = os.path.join(model_dir, "bert_config.json")
# vocab_file = os.path.join(model_dir, "vocab.txt")
token = "96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729"
vocab_file = "/home/eartemov/models/bert-base-multilingual-cased/" + token

In [68]:
from modules import BertNerData as NerData

In [69]:
data = NerData.create(train_path, valid_path, vocab_file, is_cls=True, batch_size=16)

In [70]:
len(data.tokenizer.vocab)

119547

In [71]:
import torch

In [72]:
torch.__version__

'0.4.1'

In [73]:
torch.cuda.set_device(0)

In [74]:
from modules.models.bert_models import BertBiLSTMAttnNMTJoint, BertBiLSTMAttnNCRF, BertBiLSTMAttnNCRFJoint

In [75]:
model = BertBiLSTMAttnNCRFJoint.create(
         len(data.label2idx), len(data.cls2idx), bert_config_file, init_checkpoint_pt,
         enc_hidden_dim=1024, rnn_layers=1, num_heads=6, input_dropout=0.5, nbest=11)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpk1rqwwzz
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

build CRF...


In [76]:
# model = BertBiLSTMAttnNCRF.create(
#         len(data.label2idx), bert_config_file, init_checkpoint_pt,
#        enc_hidden_dim=1024, rnn_layers=1, num_heads=6, input_dropout=0.5, nbest=11)

In [77]:
model.get_n_trainable_params()

8945684

In [78]:
from modules import NerLearner

In [79]:
num_epochs = 150
learner = NerLearner(model, data,
                     best_model_path="/home/eartemov/ae/work/models/AGRR-2019/slavic_clf.cpt",
                     # best_model_path="/home/eartemov/ae/work/models/slavic-base.cpt",
                     lr=0.0001, clip=1.0, sup_labels=data.id2label[1:],
                     t_total=num_epochs * len(data.train_dl))

In [80]:
# learner.fit(num_epochs, target_metric="f1")

In [81]:
learner.load_model()

In [82]:
from modules.data.bert_data import get_bert_data_loader_for_predict

In [83]:
dl = get_bert_data_loader_for_predict(valid_path, learner)

In [84]:
preds_res, preds_cls = learner.predict(dl)
# preds_res = learner.predict(dl)

In [85]:
from sklearn_crfsuite.metrics import flat_classification_report

In [86]:
from modules.utils.utils import bert_labels2tokens, first_choicer

In [87]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds_res)
true_tokens, true_labels = bert_labels2tokens(dl, [x.labels for x in dl.dataset])

In [88]:
len(true_tokens)

5457

In [89]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels)

#### With cls report

In [57]:
print(tokens_report)

              precision    recall  f1-score   support

       I_EVT       0.51      0.39      0.44        54
       I_LOC       0.73      0.88      0.80      1422
         I_O       0.99      1.00      0.99     72152
       I_ORG       0.81      0.64      0.72      1876
       I_PER       0.91      0.80      0.85      4121
       I_PRO       0.46      0.55      0.50       233

    accuracy                           0.97     79858
   macro avg       0.74      0.71      0.72     79858
weighted avg       0.97      0.97      0.97     79858



In [59]:
from sklearn.metrics import accuracy_score, f1_score

pl = []
for line in pred_labels:
    for p in line:
        pl.append(p)
        
tl = []
for line in true_labels:
    for p in line:
        tl.append(p)
accuracy_score(tl, pl), f1_score(tl, pl, average="macro")

(0.9743043902927697, 0.717417953930747)

In [60]:
from modules.utils.utils import tokens2spans

In [61]:
pred_spans = tokens2spans(pred_tokens, pred_labels)

In [62]:
true_spans = tokens2spans(true_tokens, true_labels)

In [63]:
from modules.utils.plot_metrics import get_bert_span_report

In [64]:
from modules.utils.utils import voting_choicer

In [65]:
print(get_bert_span_report(dl, preds_res, fn=first_choicer))

              precision    recall  f1-score   support

         PER      0.811     0.754     0.782      2846
         LOC      0.703     0.846     0.768      1477
         ORG      0.661     0.641     0.651       984
         EVT      0.375     0.444     0.407        27
         PRO      0.385     0.533     0.447       167

   micro avg      0.732     0.750     0.741      5501
   macro avg      0.587     0.644     0.611      5501
weighted avg      0.740     0.750     0.742      5501



### Predict test

In [21]:
test_path = data_dir + "test.csv"

In [ ]:
from modules import BertNerData as NerData
from modules.models.bert_models import BertBiLSTMAttnNMTJoint, BertBiLSTMAttnNCRFJoint
from modules import NerLearner

data = NerData.create(train_path, test_path, vocab_file, is_cls=True, batch_size=16)

In [17]:
data.cls2idx

{False: 1, 'O': 2, True: 0}

In [18]:
import torch

torch.cuda.set_device(4)

model = BertBiLSTMAttnNCRFJoint.create(
        len(data.label2idx), 2, bert_config_file, init_checkpoint_pt,
        enc_hidden_dim=1024, rnn_layers=1, num_heads=6, input_dropout=0.5, nbest=11)
# model = BertBiLSTMAttnNMTJoint.create(
#    len(data.label2idx), len(data.cls2idx), bert_config_file, init_checkpoint_pt,
#    enc_hidden_dim=128, rnn_layers=1, dec_embedding_dim=32, dec_hidden_dim=128, input_dropout=0.5, nbest=11)
# model = torch.nn.DataParallel(model, [2, 3])
num_epochs = 150
learner = NerLearner(model, data,
                     best_model_path="/home/aaemeljanov/models/AGRR-2019/slavic-all.cpt",
                     lr=0.0001, clip=1.0, sup_labels=data.id2label[1:],
                     t_total=num_epochs * len(data.train_dl))


build CRF...


In [20]:
learner.load_model()

In [21]:
from modules.data.bert_data import get_bert_data_loader_for_predict

dl = get_bert_data_loader_for_predict(test_path, learner)

In [23]:
preds_res, preds_cls = learner.predict(dl)

In [52]:
from modules.utils.utils import bert_labels2tokens, first_choicer

pred_tokens, pred_labels = bert_labels2tokens(dl, preds_res, fn=first_choicer)

In [53]:
from modules.utils.utils import tokens2spans

In [54]:
pred_spans = tokens2spans(pred_tokens, pred_labels)

#### To annotation df

In [16]:
import pandas as pd

In [22]:
tdf = pd.read_csv(test_path)

In [58]:
tdf["spans"] = pred_spans

In [70]:
paths = []
annotations = []

In [71]:
for path, group in tdf.groupby("path"):
    path = path.split("|")[1].replace("/home/aaemeljanov/data/", "")
    l = [r for _, r in group.iterrows()]
    spans = {}
    for r in l:
        for s in r.spans:
            if s[1] != "O":
                spans[s[0]] = s[1]
    annotation = "\n".join(["{}\tNormalForm\t{}".format(k, spans[k]) for k in spans])
    paths.append(path)
    annotations.append(annotation)

In [72]:
paths[0]

'raw/nord_stream/bg/nord_stream_bg.txt_file_1003.txt'

In [73]:
print(annotations[0])

Германия	NoralForm	LOC
Украйна	NoralForm	LOC
Швеция	NoralForm	LOC
Русия	NoralForm	LOC
Витренко	NoralForm	PER
Европа“	NoralForm	LOC
Европа	NoralForm	LOC
Дания	NoralForm	LOC
Юрий Витренко	NoralForm	PER


In [74]:
res_df = pd.DataFrame({"paths": paths, "annotations": annotations})

In [75]:
res_df.head()

,annotations,paths
0,Германия\tNoralForm\tLOC\nУкрайна\tNoralForm\t...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
1,Германия\tNoralForm\tLOC\nШвеция\tNoralForm\tL...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
2,Куртика\tNoralForm\tPER\nпоток\tNoralForm\tLOC...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
3,Варшава\tNoralForm\tLOC\nЕС\tNoralForm\tORG\nП...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
4,Германия\tNoralForm\tLOC\nУкрайна\tNoralForm\t...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...


In [76]:
res_df.to_csv("/home/aaemeljanov/data/test_pred_slavic.csv", index=False)

### Add index

In [116]:
import pandas as pd

In [117]:
tdf = pd.read_csv("test_df.csv")

In [118]:
tdf.head(10)

,Unnamed: 0,tokens,tags,lang,path,domain,cltags
0,0,германия,LOC,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,LOC-0
1,1,украйна,LOC,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,LOC-1
2,2,швеция,LOC,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,LOC-2
3,3,русия,LOC,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,LOC-3
4,4,витренко,PER,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,PER-4
5,5,европа,LOC,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,LOC-5
6,7,дания,LOC,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,LOC-6
7,8,юрий_витренко,PER,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,PER-7
8,11,бисер_петков,PER,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,PER-8
9,12,георге_иванова,PER,bg,raw/nord_stream/bg/nord_stream_bg.txt_file_100...,nord_stream,PER-9


In [119]:
res_df = pd.read_csv("/home/aaemeljanov/data/test_pred_slavic.csv")

In [120]:
res_df.head()

,annotations,paths
0,Германия\tNoralForm\tLOC\nУкрайна\tNoralForm\t...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
1,Германия\tNoralForm\tLOC\nШвеция\tNoralForm\tL...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
2,Куртика\tNoralForm\tPER\nпоток\tNoralForm\tLOC...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
3,Варшава\tNoralForm\tLOC\nЕС\tNoralForm\tORG\nП...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...
4,Германия\tNoralForm\tLOC\nУкрайна\tNoralForm\t...,raw/nord_stream/bg/nord_stream_bg.txt_file_100...


In [105]:
paths = []
annotations = []

In [109]:
for path, group in tdf.groupby("path"):
    l = [r for _, r in group.iterrows()]
    text = "{}-{}".format(path.split("/")[2], path.split(".")[-2].split("_")[-1])
    for r in l:
        if isinstance(r.tokens, str):
            tokens = r.tokens.replace("_", " ")
            tag = r.tags
            cltags = r.cltags
            normal_form = "NormalForm"
            line = "\n{}\t{}\t{}\t{}".format(tokens, normal_form, tag, cltags)
            text+=line
    path = "/home/aaemeljanov/data/annotated/" + path[4:-3] + "out"
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    paths.append(path)
    annotations.append(text)

In [113]:
len(annotations)

2394

In [115]:
len(paths)

2394

In [131]:
!ls /home/aaemeljanov/data/TESTDATA_BSNLP_2019_shared_task_pred/annotated/

nord_stream  ryanair


In [146]:
!cat /home/aaemeljanov/data/TESTDATA_BSNLP_2019_shared_task_pred/annotated/ryanair/pl/pl-ryanair-eiki.out

pl-raw/ryanair/pl/pl-ryanair-eiki
waterford	NormalForm	LOC	LOC-3008
izraelu	NormalForm	LOC	LOC-3009
cyprze	NormalForm	LOC	LOC-3010
wrocławskiego	NormalForm	LOC	LOC-3011
strachowice	NormalForm	EVT	EVT-3012
tony	NormalForm	PER	PER-3013
ryana	NormalForm	PER	PER-3014
poznania	NormalForm	LOC	LOC-3015
boeing 737-8as	NormalForm	PRO	PRO-3016
londyn-gatwick	NormalForm	LOC	LOC-3017

In [147]:
!cat /home/aaemeljanov/data/TESTDATA_BSNLP_2019_shared_task_pred/raw/ryanair/pl/pl-ryanair-eiki.txt

ryanair wiki
pl
1970-01-01








W 2018 roku Ryanair uruchomił w Polsce linie czarterowe Ryanair Sun. Pierwszy lot odbył się 26 kwietnia 2018, z Poznania do Zakintos.
